Assignment 1

In [303]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer

from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

In [304]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Loan_approval\train.csv')
df.set_index('Loan_ID', inplace=True)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [305]:
df_test = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Loan_approval\test.csv')
df_test.set_index('Loan_ID', inplace=True)


In [306]:
X = df.drop(columns=['Loan_Status'])
y = df['Loan_Status'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

In [307]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491 entries, LP002301 to LP001900
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             480 non-null    object 
 1   Married            488 non-null    object 
 2   Dependents         477 non-null    object 
 3   Education          491 non-null    object 
 4   Self_Employed      468 non-null    object 
 5   ApplicantIncome    491 non-null    int64  
 6   CoapplicantIncome  491 non-null    float64
 7   LoanAmount         474 non-null    float64
 8   Loan_Amount_Term   481 non-null    float64
 9   Credit_History     447 non-null    float64
 10  Property_Area      491 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 46.0+ KB


In [308]:
X_train.isnull().sum()

Gender               11
Married               3
Dependents           14
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           17
Loan_Amount_Term     10
Credit_History       44
Property_Area         0
dtype: int64

In [309]:
X_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP002301,Female,No,0,Graduate,Yes,7441,0.0,194.0,360.0,1.0,Rural
LP001824,Male,Yes,1,Graduate,No,2882,1843.0,123.0,480.0,1.0,Semiurban
LP001964,Male,Yes,0,Not Graduate,No,1800,2934.0,93.0,360.0,0.0,Urban
LP001560,Male,Yes,0,Not Graduate,No,1863,1041.0,98.0,360.0,1.0,Semiurban
LP001091,Male,Yes,1,Graduate,NaN,4166,3369.0,201.0,360.0,NaN,Urban


In [310]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

object_imputer = SimpleImputer(strategy='constant', fill_value='unknown').set_output(transform='pandas')
integer_imputer = SimpleImputer(strategy='median').set_output(transform='pandas')


ct_imputer = make_column_transformer(
    (object_imputer, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    (integer_imputer, make_column_selector(dtype_include=[
        'number',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')
 
ct_ohe = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=[
        'object',
        'category',
    ])),
    (ohe, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')



In [311]:
X_train_imp = ct_imputer.fit_transform(X_train)
X_test_imp = ct_imputer.transform(X_test)

In [312]:
# Fit and transform X_train using the pipeline
X_train_transformed = ct_ohe.fit_transform(X_train_imp)

# Transform X_test using the already-fitted pipeline
X_test_transformed = ct_ohe.transform(X_test_imp)

In [313]:
lr = LogisticRegression()

pipe = Pipeline([
    #('ct_imputer', ct_imputer),
    # ('ct_ohe', ct_ohe),
    ('lr', lr),
])

params = {
    "lr__solver": [
        'lbfgs',
        'liblinear',
        'newton-cg',
        'newton-cholesky',
        'sag',
        'saga',
   ],
    "lr__multi_class": [
        "ovr",
        "multinomial"
    ],
    "lr__C" : np.linspace(0.001, 10, 10)
}


In [314]:
kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

gcv = GridSearchCV(
    estimator=pipe,
    param_grid=params,
    cv=kfold,
)



In [315]:
# Now perform model training and evaluation
gcv.fit(X_train_transformed, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('lr', LogisticRegression())]),
             param_grid={'lr__C': array([1.000e-03, 1.112e+00, 2.223e+00, 3.334e+00, 4.445e+00, 5.556e+00,
       6.667e+00, 7.778e+00, 8.889e+00, 1.000e+01]),
                         'lr__multi_class': ['ovr', 'multinomial'],
                         'lr__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']})

In [317]:
print("Logistic Reg. best params: ", gcv.best_params_)
print("Logistic Reg. best score: ",gcv.best_score_)

Logistic Reg. best params:  {'lr__C': 5.556, 'lr__multi_class': 'ovr', 'lr__solver': 'liblinear'}
Logistic Reg. best score:  0.8126571840857555


In [318]:
# Predict on the transformed test set
best_model = gcv.best_estimator_
y_pred = best_model.predict(X_test_transformed)
print("Logistic Reg. accuracy_score:", accuracy_score(y_test, y_pred))

Logistic Reg. accuracy_score: 0.7967479674796748


In [319]:
y_pred_proba = gcv.predict_proba(X_test_transformed)[:, 1]
print("Logistic Reg. roc_auc_score:", roc_auc_score(y_test, y_pred_proba))

Logistic Reg. roc_auc_score: 0.695046439628483


(123,)